## deltaLens Data Compare Sample

In [9]:
#imports

from delta_lens.deltaLens import DeltaLens, EntityComparer
from delta_lens.config import Config, Entity, Side, Transform, Defaults,ReferenceDataset, load_config
from delta_lens.sqliteExport import *
from delta_lens.csvExport import *
from datetime import date

In [2]:
#config

config = load_config("data/compare.config.json")


In [3]:
runName = f'tradeCompare_{date.today().strftime("%Y-%m-%d")}'

comparator = DeltaLens(runName,config)

comparator.execute(continue_on_error=True)



#### entity_compare_results
Table containing summary of each entity comparison

In [4]:
comparator.con.execute("SELECT * FROM  entity_compare_results").fetch_df()

,entity,rows_left,rows_right,rows_fully_matched,error_text,success
0,trade,42999117,43000000,42113850,None,1


#### [entitiy]_compare_field_summary
Entity specific table containing summary of field comparison

In [5]:
comparator.con.execute("SELECT * FROM trade_compare_field_summary").fetch_df()

,field,total,matches,match_percentage
0,trade_type,42999117,42999117.0,100.000000
1,quantity,42999117,42999117.0,100.000000
2,price,42999117,42113850.0,97.940002
3,symbol,42999117,42999117.0,100.000000
4,timestamp,42999117,42999117.0,100.000000


#### [entity]_compare
Entity specific table containing details of comparison

In [6]:
comparator.con.execute(" WITH CTE as (SELECT * FROM  trade_compare WHERE _full_match = 0) SELECT * FROM CTE USING SAMPLE 5;").fetch_df()

,trade_id,trade_type_left,trade_type_right,trade_type_match,quantity_left,quantity_right,quantity_match,price_left,price_right,price_match,symbol_left,symbol_right,symbol_match,timestamp_left,timestamp_right,timestamp_match,_exists_left,_exists_right,_full_match
0,3302447,BUY,BUY,True,1911,1911,True,586.75,320.37,False,BAC,BAC,True,2024-06-06 20:34:26,2024-06-06 20:34:26,True,True,True,False
1,17088349,SELL,SELL,True,26,26,True,433.52,259.64,False,META,META,True,2024-06-24 20:36:08,2024-06-24 20:36:08,True,True,True,False
2,24536981,BUY,BUY,True,1461,1461,True,329.42,93.86,False,AMZN,AMZN,True,2024-03-05 20:37:03,2024-03-05 20:37:03,True,True,True,False
3,24544036,SELL,SELL,True,6223,6223,True,45.60,822.31,False,AAPL,AAPL,True,2024-07-13 20:37:03,2024-07-13 20:37:03,True,True,True,False
4,24630286,BUY,BUY,True,2845,2845,True,904.55,500.03,False,BAC,BAC,True,2024-04-26 20:37:03,2024-04-26 20:37:03,True,True,True,False


#### Export to sqlite
Function to export above tables to on-disk format. Sqlite is a popular single file database, well supported in many languages,  for which lots of query tools exist. See [Datasette](https://datasette.io/) package.

In [7]:
export_to_sqlite(comparator.con, f'{runName}.sqlite')

'tradeCompare_2025-02-15.sqlite'

In [10]:
export_to_csv_archive(comparator.con,f'{runName}.tar.gz' )

'tradeCompare_2025-02-15.tar.gz'